# Text Summarization with T5 Model

In this notebook you will perform text summarizaiton task on the XSum dataset using the T5 (Text-to-Text Transfer Transformer).

Text summarization is an NLP task that involves extracting the main points and important information from a long piece of text while creating a shorter version of it without losing any of the meaning. The XSum (Extreme Summarization) dataset is a dataset containing news articles from the BBC and short, concise summaries of these articles.

The input to your model will be text data from the XSum dataset, and the output will be the summarized text. The goal is to fine-tune the T5 model using this dataset to produce high-quality summaries of news articles.



![**Each NLP problem as a “text-to-text” problem** - input: text, output: text](https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png)



### [Text Summarization with T5 Model](#scrollTo=rEJBSTyZIrIb)

[What is T5?](#scrollTo=2bvevT7-w9z1)

>[Different T5 models](#scrollTo=D2Qohq12w_6A)

>[2 Types of Summary Generation Mechanisms](#scrollTo=PhhXP2UgxCSf)

>>[1)Extractive Summarizaiton](#scrollTo=B8BbYI5YxEXy)

>>[1)Abstractive Summarizaiton](#scrollTo=NIA6DgC1xIk7)

[1) Import Libraries and Set up GPU](#scrollTo=DsRrkZx9xOwB)

[2) Exploratory Data Analysis and Preprocessing](#scrollTo=p7Ko5WCDyD6P)

>[XSum (Extreme Summarization) Dataset](#scrollTo=4tPKytq9A6Uv)

[3) Initialize The Pre-trained T5 Model and Load The Tokenizer](#scrollTo=rmCGQOH-42v8)

[4) Tokenize the Dataset](#scrollTo=anMrcgM8NIvG)

[5) Load The Pre-Trained T5 Model](#scrollTo=hNNNbcaA8E8_)

[What is ROUGE Metric ?](#scrollTo=ki_djYN7UjqT)

>[Different ROUGE Metric Types](#scrollTo=q5F3zcXsVDBj)

[6) Train and Evaluate Model](#scrollTo=12lNkB9VNku8)

[7) Inference from Fine-Tuned Model](#scrollTo=rQDfC7HtL8nC)



# What is T5?

The T5  model is a transformer based model introduced by Google Research in 2019. T5 is based on the Transformer architecture and aims to handle most natural language processing tasks in a single **text-to-text** framework. This means that the model takes a text as input and produces a text again.

Here is the link for research paper : [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/abs/1910.10683v4)

For more detailed information about the T5 model, see: [T5](https://huggingface.co/docs/transformers/model_doc/t5#t5)


### Different T5 models

| Model         | Encoder Layers | Decoder Layers | Hidden Size | FFN Size | Attention Heads | Parameters   |
|---------------|----------------|----------------|-------------|----------|-----------------|--------------|
| T5 Small      | 6              | 6              | 512         | 2048     | 8               | 60 million   |
| T5 Base       | 12             | 12             | 768         | 3072     | 12              | 220 million  |
| T5 Large      | 24             | 24             | 1024        | 4096     | 16              | 770 million  |
| T5 3B         | 24             | 24             | 1024        | 4096     | 32              | 3 billion    |
| T5 11B        | 24             | 24             | 1024        | 4096     | 64              | 11 billion   |

### 2 Types of Summary Generation Mechanisms

![**nsp](https://lh4.googleusercontent.com/83a1_AZrS1EoCxczzF_x6bObhj-nX1t81SfOC-NprCQaDbHQ7xgcTcefuIlYff-i9Gajnl8eegWlfqvseE_mHP4j8mTquhmN8-HNztC7sc_gRGbTtvJSBXvactWAe85Raky0IXNF1zQycFBcJMLk8KE)

> ### 1)Extractive Summarizaiton

Extractive summarization is a technique used to generate summaries by selecting key sentences, phrases, or sections **directly from the source text**. Extractive summarizaiton can be based on ML methods, statistical methods such as **TF-IDF (Term Frequency-Inverse Document Frequency)** or graph based methods such as **TextRank** and **LexRank**.


> ### 1)Abstractive Summarizaiton

Abstractive summarization is a technique used to generate summaries by **creating new sentences that capture the essence of the original text**. Unlike extractive summarization, which selects and compiles sentences directly from the source text, abstractive summarization involves paraphrasing and rephrasing the information to produce a more coherent and concise summary.

![**Extractive vs Abstractive](https://devopedia.org/images/article/261/2509.1582303438.png)




You will perform **abstractive summarizaiton** on the data with the T5 model in this notebook.



---



## 1)Import Libraries and Set up GPU

In [ ]:
! pip install datasets evaluate transformers rouge-score nltk session-info -q

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import datasets
import random
import nltk
nltk.download('punkt')

from datasets import load_dataset, DatasetDict
from evaluate import load
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
# Check if cuda is avaliable and set gpu device.

!nvidia-smi

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## 2) Exploratory Data Analysis and Preprocessing

### XSum (Extreme Summarization) Dataset

Created by the BBC articles, the XSum dataset is designed for the task of extreme summarization, which aims to generate very short summaries that capture the essence of the original document. Dataset contains around 226,711 news articles, each paired with a single-sentence summary.

*Narayan, S., Cohen, S. B., & Lapata, M. (2018). Don't give me the details, just the summary! topic-aware convolutional neural networks for extreme summarization. https://arxiv.org/abs/1808.08745*

Dataset: https://huggingface.co/datasets/EdinburghNLP/xsum


In [ ]:
# Import the XSum dataset with Hugginface's load_dataset function (https://huggingface.co/docs/datasets/loading). Then examine the data set.

# YOUR CODE STARTS HERE
dataset = load_dataset(None,
                       trust_remote_code=None)

# YOUR CODE ENDS HERE

dataset

In [ ]:
# Create a smaller training dataset with random selections from the dataset for faster training times.

# YOUR CODE STARTS HERE

sample_ratio = None

# YOUR CODE ENDS HERE
small_dataset = DatasetDict({
    'train': dataset['train'].shuffle(seed=42).select(range(int(len(dataset['train']) * sample_ratio))),
    'validation': dataset['validation'].shuffle(seed=42).select(range(int(len(dataset['validation']) * sample_ratio))),
    'test': dataset['test'].shuffle(seed=42).select(range(int(len(dataset['test']) * sample_ratio)))
})

small_dataset

In [ ]:
# Examine examples from the dataset.

small_dataset["train"][0]

## 3) Initialize The Pre-trained T5 Model and Load The Tokenizer

In [ ]:
# Initialize the T5 model and create the tokenizer object with Transformer's AutoTokenizer class.

# YOUR CODE STARTS HERE

model_id = None
tokenizer = AutoTokenizer.from_pretrained(model_id)

# YOUR CODE ENDS HERE

In [ ]:
# Check the tokenizer components.

tokenizer

In [ ]:
# An example output of the T5 tokenizer.

tokenizer(["Hello, this one sentence!", "This is another sentence."])

If you are using one of the five available (checkpoint) versions of the T5 model, the input system must have a prefix of **"summarize:"** attached in order for the model to work in the correct role.



In [ ]:
if model_id in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:

# YOUR CODE STARTS HERE

    prefix = None
else:
    prefix = ""

# YOUR CODE STARTS HERE

## 4) Tokenize the Dataset

Edit model inputs with `preprocess_function` to preprocess input data and target summaries for the text summarization model.

In [ ]:
# YOUR CODE STARTS HERE

max_input_length = None
max_target_length = None

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs,
                             max_length=max_input_length,
                             truncation=None)

    labels = tokenizer(text_target=examples["summary"],
                       max_length=max_target_length,
                       truncation=None)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# YOUR CODE ENDS HERE

In [ ]:
preprocess_function(small_dataset['train'][:2])

In [ ]:
# Tokenize the data by applying the preprocess function to all data.

# YOUR CODE STARTS HERE

tokenized_datasets = small_dataset.map(None,
                                      batched=None)

# YOUR CODE ENDS HERE
tokenized_datasets

## 5) Load The Pre-Trained T5 Model

Load Pre-trained model with '[AutoModelForSeq2SeqLM.from_pretrained](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSeq2SeqLM)'.

In [ ]:
# YOUR CODE STARTS HERE

model = None

# YOUR CODE ENDS HERE

model.to(device) # Move the model to the GPU. Check the model architecture.

Perform fine tuning using Hugginface's Trainer class. `Seq2SeqTrainer` and `Seq2SeqTrainingArguments` inherit from the Trainer and TrainingArgument classes and they're adapted for training models for sequence-to-sequence tasks such as summarization or translation.

Set training arguments using  [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/v4.43.3/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments).


In [ ]:
# YOUR CODE STARTS HERE

batch_size = 8 # recommended 8
model_name = model_id.split("/")[-1]
save_directory = f"{model_name}-finetuned-xsum"
args = Seq2SeqTrainingArguments(
    output_dir = save_directory,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    gradient_accumulation_steps=2
)

# YOUR CODE ENDS HERE

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Set up a special type of data collector with [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForSeq2Seq) that will fill not only the inputs but also the labels to the maximum length in the stack.

In [ ]:
# YOUR CODE STARTS HERE

data_collator = DataCollatorForSeq2Seq(tokenizer = None,
                                       model = None)

# YOUR CODE ENDS HERE

## What is ROUGE Metric ?

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metrics are a set of metrics used to evaluate the quality of automated text summaries and machine translations. ROUGE metrics evaluate how much of the summary overlaps with the reference summary. ROUGE calculates these overlaps by comparing word n-grams, word sequences, and word pairs.

[Original research paper for ROUGE metric](https://aclanthology.org/W04-1013/)



> #### Different ROUGE Metric Types

- ROUGE-N: It measures the overlap between the reference summary and the automatic summary on an **n-gram** basis (strings of words of length n).

    - ROUGE-1: **Unigram (1-gram)** based scoring.
    - ROUGE-2: **Bigram (2-gram)** based scoring
- ROUGE-L: It measures based on the **Longest Common Subsequence (LCS).**
- ROUGE-W: It is a **weighted version of ROUGE-L**. It gives more weight to longer common subsequences.
- ROUGE-S: It measures the **overlap of word pairs**. It does not take into account the number of words between them.


In [ ]:
# Load rouge score with metric function from evaluate library.

# YOUR CODE STARTS HERE

metric = load(None)

# YOUR CODE ENDS HERE
metric

Check out some examples of using the ROUGE metric.



In [ ]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = metric.compute(predictions=predictions,
                         references=references)
print(results)

In [ ]:
predictions = ["hello there", "general kenobi"]
references = [["hello", "there"], ["general kenobi", "general yoda"]]
results = metric.compute(predictions=predictions,
                        references=references)
print(results)

Compute ROUGE metrics using the model's predictions and actual labels (references) with the `compute_metrics` function.

In [ ]:
# The following function calculates summarization metrics (ROUGE) to evaluate the summarization performance of the model, decodes the predictions and labels, converts them into text format, separates them into sentences and calculates the metrics

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions,
                                           skip_special_tokens=True)  # Converts the predicted tokens back to text.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # Replaces -100 values ​​in labels with pad_token_id (usually this is for tokens that the model does not include in learning).
    decoded_labels = tokenizer.batch_decode(labels,
                                            skip_special_tokens=True) # Converts actual summaries from tokens to text.


    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 6) Train and Evaluate Model

Initialize the trainer using `Seq2SeqTrainer` from the Hugginface trainer class.

See for more detailed information:[Seq2SeqTrainer](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainer).

In [ ]:
# YOUR CODE STARTS HERE

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=None,
    data_collator=None,
    tokenizer=None,
    compute_metrics=None
)

# YOUR CODE ENDS HERE

In [ ]:
trainer.train()

In [ ]:
# Load the model you fine tuned.

trainer.save_model(save_directory)

## 7) Inference from Fine-Tuned Model

In [ ]:
# Load the model you fine tuned and inference on the samples in the test set.

ft_model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)
ft_tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [ ]:
# Run the summarize_text function, which prepares the input text and produces the summary.

def summarize_text(text):
    inputs = ft_tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    summary_ids = ft_model.generate(
        inputs,
        max_length=50,
        num_beams=5,

    )
    return ft_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Get samples from test dataset.

test_text = small_dataset['test'][50]['document']
test_text

In [ ]:
# Get the summary for the sample text of your choice.

test_summarize = summarize_text(test_text)
test_summarize

In [ ]:
import session_info
session_info.show()